In [3]:
import pytz
import pandas as pd
pd.set_option("display.max_rows", 5000)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:

games = pd.read_csv("/kaggle/input/mlb-pitch-data-20152018/games.csv")

tzmap = {
    'ana': "America/Los_Angeles", # angels
    'ari': "US/Arizona",
    'atl': "US/Eastern",
    'bal': "US/Eastern",
    'bos': "US/Eastern",
    'cha': "US/Central",  # white sox
    'chn': "US/Central",  # cubs
    'cin': "US/Eastern",
    'cle': "US/Eastern",
    'col': "US/Mountain",
    'det': "US/Eastern",
    'hou': "US/Central",
    'kca': "US/Central",
    'lan': "America/Los_Angeles",  # dodgers
    'mia': "US/Eastern",
    'mil': "US/Central",
    'min': "US/Central",
    'nya': "US/Eastern",
    'nyn': "US/Eastern",
    'oak': "US/Pacific",
    'phi': "US/Eastern",
    'pit': "US/Eastern",
    'sdn': "US/Pacific",
    'sea': "US/Pacific",
    'sfn': "US/Pacific",  # giants
    'sln': "US/Central",  # cards
    'tba': "US/Eastern",  # rays
    'tex': "US/Central",
    'tor': "America/Toronto",
    'was': "US/Eastern",
}

games["dtdate"] = pd.to_datetime(games["date"])
games["day_of_week"] = games["dtdate"].dt.weekday
games["weekend"] = games["day_of_week"] > 5
games["start_datetime"] = games.apply(lambda d: pd.to_datetime("{} {}".format(d["date"], d["start_time"])).tz_localize(tzmap.get(d["home_team"], None)), axis=1)
games["start_datetime"].apply(lambda o: type(o)).unique()

In [5]:
games["localized_hour"] = games["start_datetime"].apply(lambda d: d.tz_convert("US/Eastern").hour)
games[["home_team", "start_datetime", "localized_hour"]]

In [6]:
games[~games["weekend"]].groupby("localized_hour").count().sort_index()["attendance"].plot(kind="bar", figsize=(15, 10), title="Weekdays")

In [7]:
games[games["weekend"]].groupby("localized_hour").count().sort_index()["attendance"].plot(kind="bar", figsize=(15, 10), title="Weekends")